### Laboratório Escalonamento 2

Alunos: Max Souza, Noah Diunkz


### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

In [1024]:
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada,bilhetes=0):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria agora 
        self.chegada = tempoChegada
        self.bilhetes = bilhetes


    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    

class escalonador_LOTERIA(object):
    def __init__(self, vprontos=[], total_tickets=400):
        self.prontos = vprontos 
        self.name = "escalonador_LOTERIA"
        self.total_tickets = total_tickets
        self._total_tickets_aux = total_tickets
        self._io_blocked_list = []

    def pronto(self, Processo, io=False, born=False):
        self.prontos.append(Processo)

    def proximo(self):
        total = 0
        number = random.randint(0, self._total_tickets_aux)

        for i in range(len(self.prontos)):
            total += self.prontos[i].bilhetes
            if (number <= total):
                return self.prontos.pop(i)

        return None

    def io_blocked(self):
        i, lim = 0, len(self._io_blocked_list)

        while (i < lim):
            if (self._io_blocked_list[i][1] == 0):
                self.pronto(self._io_blocked_list[i][0], io=True)
                del self._io_blocked_list[i]
                lim -= 1
            else:
                i += 1

    def io_blocked_add(self, Process, block_time):
        self._io_blocked_list.append([Process, block_time + 1])
        

class escalonador_STRIDE(object):
    def __init__(self,vprontos=[]):
        self.prontos = vprontos 
        
    
    def pronto(self,Processo): 
        if(Processo not in self.prontos):
            self.prontos.append(Processo)
        
        
    def proximo(self):
        for i in range(len(self.prontos)):
            if self.prontos[i].tam > 0:
                return self.prontos[i]
            
        for i in range(len(self.prontos)):
            self.prontos[i].bilhetes += 1
            if self.prontos[i].bilhetes >= self.prontos[i].tam:
                self.prontos[i].bilhetes = 0
                return self.prontos[i]
            

class escalonador_MLFQ(object):
    def __init__(self, vprontos=[], lines=7, boost=50, quantum_lines=None):
        self.prontos = vprontos 
        self.name = "escalonador_MLFQ"
        self.filas = []
        self.boost = boost
        self._auxboost = boost
        self.lines = lines
        self.quantum = quantum_lines
        self.filas.append(vprontos)
        if (self.lines >= 1):
            for _ in range(self.lines - 1):
                self.filas.append([])

    def pronto(self, Processo, io=False, born=False):
        if (born == True):
            self.filas[0].append(Processo)
            return
        if (Processo.prio < (self.lines - 1)):
            if (io == False):
                Processo.prio += 1

            self.filas[Processo.prio].append(Processo)

    def proximo(self):
        for item in self.filas:
            if (len(item) != 0):
                return item.pop(0)

        return None

    def boost_line(self, time_spent):
        """ Depois de um tempo S definido, todos os processos que estão em filas de menor prioridade
        são realocados para a fila de máxima prioridade."""

        if (self.boost - time_spent <= 0):
            self.boost = self._auxboost
            for i in range(1, len(self.filas)):
                for j in self.filas[i]:
                    j.prio = 0

                self.filas[0] += self.filas[i]
                self.filas[i] = []
        else:
            self.boost -= time_spent


### Simulador Simples


In [1025]:
def simulador_simples_LOTERIA_STRIDE(escalonador_escolhido, nprocs, nomes, chanceio, tamanho, bilhetes, quantum, seed):

    procs = []
    for i in range(nprocs):
        procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0, bilhetes[i])) # cria uma lista procs de Processos inserindo bilhetes


    tempo = 0
    tempoE = []
    tempoR = []
    indice = 0
    total = tamanho.sum()
    random.seed(seed)


    if escalonador_escolhido == "escalonador_LOTERIA":
        escalonador = escalonador_LOTERIA(procs) #troque escalonador pelo seu escalonador
    if escalonador_escolhido == "escalonador_STRIDE":
        escalonador = escalonador_STRIDE(procs) #troque escalonador pelo seu escalonador
    if escalonador_escolhido not in ("escalonador_LOTERIA", "escalonador_STRIDE"):
        print("Escalonador incorreto. Escolha escalonador_LOTERIA ou escalonador_STRIDE")


    #print("Escalonamento por Loteria")

    while total > 0:
        p = escalonador.proximo()
        if (p is not None):
            rodou, _ = p.roda(quantum) #adicione quantum como parâmetro, por enquanto nao temos E/S

            if len(tempoE) == 0:
                tempoE.append(rodou)
                tempoR.append(indice)
            else:
                tempoE.append(rodou + tempoE[indice])
                tempoR.append(tempoE[indice])
                indice += 1

            if (p.tam > 0):
                escalonador.pronto(p)
            total -= rodou
            tempo += rodou

        else:
            #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
            tempo += 1

    print("Tempo de execução:", sum(tempoE) / nprocs)
    print("Tempo de resposta:", sum(tempoR) / nprocs)
    print("Tempo médio de execução:", sum(tempoE) / len(tempoE))
    print("Tempo médio de resposta:", sum(tempoR) / len(tempoR))


### Simulador Maior

In [1026]:
def simulador_maior_LOTERIA_STRIDE(escalonador_escolhido, nprocs, nomes, chanceio, tamanho, bilhetes, quantum, seed):

    procs = []
    for i in range(nprocs):
        procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0, bilhetes[i])) # cria uma lista procs de Processos inserindo bilhetes
    
    tempo = 0
    tempoE = []
    tempoR = []
    indice = 0
    maximo = 100
    contaProc = 1
    chanceNovoProcesso = 60
    chanceIo = 10
    minTime = 4
    maxTime = 10
    tempo = 0
    tempoE = []
    tempoR = []
    indice = 0
    total = tamanho.sum()
    random.seed(seed) # Seed

    if escalonador_escolhido == "escalonador_LOTERIA":
        escalonador = escalonador_LOTERIA(procs) #troque escalonador pelo seu escalonador
    if escalonador_escolhido == "escalonador_STRIDE":
        escalonador = escalonador_STRIDE(procs) #troque escalonador pelo seu escalonador
    if escalonador_escolhido not in ("escalonador_LOTERIA", "escalonador_STRIDE"):
        print("Escalonador incorreto. Escolha escalonador_LOTERIA ou escalonador_STRIDE")
        

    while total > 0:
        if(tempo<maximo and random.randint(1,100)<chanceNovoProcesso):
            p = Processo('P'+str(contaProc),random.randint(1,chanceIo),random.randint(minTime,maxTime),random.randint(1,100),tempo)
            print("Processo",p.nome," chegou no tempo",tempo)
            escalonador.pronto(p)
            contaProc+=1

        p = escalonador.proximo()
        if (p is not None):
            rodou, _ = p.roda(quantum) #adicione quantum como parâmetro, por enquanto nao temos E/S

            if len(tempoE) == 0:
                tempoE.append(rodou)
                tempoR.append(indice)
            else:
                tempoE.append(rodou + tempoE[indice])
                tempoR.append(tempoE[indice])
                indice += 1

            if (p.tam > 0):
                escalonador.pronto(p)
            total -= rodou
            tempo += rodou

        else:
            #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
            tempo += 1

    print("Tempo de execução:", sum(tempoE) / nprocs)
    print("Tempo de resposta:", sum(tempoR) / nprocs)
    print("Tempo médio de execução:", sum(tempoE) / len(tempoE))
    print("Tempo médio de resposta:", sum(tempoR) / len(tempoR))


### Sobre Loteria


Questão 1 - Rode a simulação mais simples com apenas dois processos, um com 100 tickets e outro com 1, com tempo 100 para terminar. O que aconteceu? O segundo processo conseguiu rodar alguma vez? Testando com outras sementes o resultado se manteve?

Resposta:

Com 100 bilhetes para A e 1 bilhete para B, aconteceu que o processo A foi selecionado para ser executado primeiro, pois tinha mais bilhetes e rodou por 100 timeslices até terminar, afinal, sempre tinha mais bilhetes que B. Só ao final dos processos de A que o processo B conseguiu ser executado. Portanto, a resposta é sim, o segundo processo conseguiu rodar alguma vez.

Testando com outras sementes, por exemplo 150, o resultado foi diferente. Notamos que dentre alguns processos de A, B foi executado. Mas isso já era esperado, pois o algoritmo de escalonamento por loteria é um algoritmo probabilístico. Nesse contexto, isso significa que, embora o processo A tenha muito mais bilhetes do que o processo B, ainda há uma pequena chance de que o processo B seja escolhido para executar.

Exemplo com seed=150
```
...
A  rodou por  1  timeslice, faltam  66
B  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  65
...
```

In [1027]:
# Loteria Questão 1

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 1] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 1
seed = 0
simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
B  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslice, faltam  76
A  rodou por  1  timeslic

In [1028]:
# Loteria Questão 1: Outras sementes

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 1] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 1
seed = 150

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslice, faltam  76
A  rodou por  1  timeslice, faltam  75
A  rodou por  1  timeslic

Questão 2 - Rode a simulação simples com dois processos com 100 tickets e 100 timeslices. Calcule a Unfairness para quantum=2. Repita a operação para quantum 10, 20, 50 e 100. O que aconteceu?

In [1029]:
# Loteria Questão 2: Quantum=2 

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100] # tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 2
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

B  rodou por  2  timeslice, faltam  98
A  rodou por  2  timeslice, faltam  98
A  rodou por  2  timeslice, faltam  96
A  rodou por  2  timeslice, faltam  94
A  rodou por  2  timeslice, faltam  92
B  rodou por  2  timeslice, faltam  96
A  rodou por  2  timeslice, faltam  90
B  rodou por  2  timeslice, faltam  94
B  rodou por  2  timeslice, faltam  92
A  rodou por  2  timeslice, faltam  88
B  rodou por  2  timeslice, faltam  90
A  rodou por  2  timeslice, faltam  86
A  rodou por  2  timeslice, faltam  84
B  rodou por  2  timeslice, faltam  88
B  rodou por  2  timeslice, faltam  86
B  rodou por  2  timeslice, faltam  84
B  rodou por  2  timeslice, faltam  82
A  rodou por  2  timeslice, faltam  82
B  rodou por  2  timeslice, faltam  80
B  rodou por  2  timeslice, faltam  78
B  rodou por  2  timeslice, faltam  76
A  rodou por  2  timeslice, faltam  80
A  rodou por  2  timeslice, faltam  78
B  rodou por  2  timeslice, faltam  74
A  rodou por  2  timeslice, faltam  76
A  rodou por  2  timeslic

Calculo unfairness


Fazer  o calculo

In [1030]:
# Loteria Questão 2: Quantum=10 

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100]
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 10
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

B  rodou por  10  timeslice, faltam  90
A  rodou por  10  timeslice, faltam  90
A  rodou por  10  timeslice, faltam  80
A  rodou por  10  timeslice, faltam  70
A  rodou por  10  timeslice, faltam  60
B  rodou por  10  timeslice, faltam  80
A  rodou por  10  timeslice, faltam  50
B  rodou por  10  timeslice, faltam  70
B  rodou por  10  timeslice, faltam  60
A  rodou por  10  timeslice, faltam  40
B  rodou por  10  timeslice, faltam  50
A  rodou por  10  timeslice, faltam  30
A  rodou por  10  timeslice, faltam  20
B  rodou por  10  timeslice, faltam  40
B  rodou por  10  timeslice, faltam  30
B  rodou por  10  timeslice, faltam  20
B  rodou por  10  timeslice, faltam  10
A  rodou por  10  timeslice, faltam  10
B  rodou por  10  timeslice, faltam  0
A  rodou por  10  timeslice, faltam  0
Tempo de execução: 1050.0
Tempo de resposta: 950.0
Tempo médio de execução: 105.0
Tempo médio de resposta: 95.0


Calculo unfairness


Fazer  o calculo

In [1031]:
# Loteria Questão 2: Quantum=20 

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100]
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 20
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

B  rodou por  20  timeslice, faltam  80
A  rodou por  20  timeslice, faltam  80
A  rodou por  20  timeslice, faltam  60
A  rodou por  20  timeslice, faltam  40
A  rodou por  20  timeslice, faltam  20
B  rodou por  20  timeslice, faltam  60
A  rodou por  20  timeslice, faltam  0
B  rodou por  20  timeslice, faltam  40
B  rodou por  20  timeslice, faltam  20
B  rodou por  20  timeslice, faltam  0
Tempo de execução: 550.0
Tempo de resposta: 450.0
Tempo médio de execução: 110.0
Tempo médio de resposta: 90.0


Calculo unfairness


Fazer  o calculo

In [1032]:
# Loteria Questão 2: Quantum=50

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100]
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 50
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

B  rodou por  50  timeslice, faltam  50
A  rodou por  50  timeslice, faltam  50
A  rodou por  50  timeslice, faltam  0
B  rodou por  50  timeslice, faltam  0
Tempo de execução: 250.0
Tempo de resposta: 150.0
Tempo médio de execução: 125.0
Tempo médio de resposta: 75.0


Calculo unfairness


Fazer  o calculo

In [1033]:
# Loteria Questão 2: Quantum=100

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100]
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 100
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

B  rodou por  100  timeslice, faltam  0
A  rodou por  100  timeslice, faltam  0
Tempo de execução: 150.0
Tempo de resposta: 50.0
Tempo médio de execução: 150.0
Tempo médio de resposta: 50.0


Calculo unfairness


Fazer  o calculo

Questão 3 - Rode novamente dois processos com 100 tickets no simulador simples e quantum=2. Calcule a unfairness conforme o tamanho dos processos aumenta e faça um gráfico similar ao dos slides.

In [1034]:
# Loteria Questão 2: Quantum=2 

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 2
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)


B  rodou por  2  timeslice, faltam  98
A  rodou por  2  timeslice, faltam  98
A  rodou por  2  timeslice, faltam  96
A  rodou por  2  timeslice, faltam  94
A  rodou por  2  timeslice, faltam  92
B  rodou por  2  timeslice, faltam  96
A  rodou por  2  timeslice, faltam  90
B  rodou por  2  timeslice, faltam  94
B  rodou por  2  timeslice, faltam  92
A  rodou por  2  timeslice, faltam  88
B  rodou por  2  timeslice, faltam  90
A  rodou por  2  timeslice, faltam  86
A  rodou por  2  timeslice, faltam  84
B  rodou por  2  timeslice, faltam  88
B  rodou por  2  timeslice, faltam  86
B  rodou por  2  timeslice, faltam  84
B  rodou por  2  timeslice, faltam  82
A  rodou por  2  timeslice, faltam  82
B  rodou por  2  timeslice, faltam  80
B  rodou por  2  timeslice, faltam  78
B  rodou por  2  timeslice, faltam  76
A  rodou por  2  timeslice, faltam  80
A  rodou por  2  timeslice, faltam  78
B  rodou por  2  timeslice, faltam  74
A  rodou por  2  timeslice, faltam  76
A  rodou por  2  timeslic

Questão 4 - Rode o simulador maior com três tipos de processo possíveis: um com 10 tickets, um com 20 e um com 50. O que aconteceu com os que receberam menos tickets? Eles tiveram chance de rodar?

In [1035]:
# Loteria Questão 2: Quantum=2 

nprocs = 3
nomes = ['A', 'B', 'C']
chanceio = [0, 0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100,100]) # Timeslices
billhetes = [10, 20, 50] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 1
seed = 0

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)


B  rodou por  1  timeslice, faltam  99
B  rodou por  1  timeslice, faltam  98
B  rodou por  1  timeslice, faltam  97
C  rodou por  1  timeslice, faltam  99
C  rodou por  1  timeslice, faltam  98
C  rodou por  1  timeslice, faltam  97
C  rodou por  1  timeslice, faltam  96
C  rodou por  1  timeslice, faltam  95
C  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  99
B  rodou por  1  timeslice, faltam  96
C  rodou por  1  timeslice, faltam  93
C  rodou por  1  timeslice, faltam  92
C  rodou por  1  timeslice, faltam  91
C  rodou por  1  timeslice, faltam  90
C  rodou por  1  timeslice, faltam  89
C  rodou por  1  timeslice, faltam  88
B  rodou por  1  timeslice, faltam  95
C  rodou por  1  timeslice, faltam  87
C  rodou por  1  timeslice, faltam  86
B  rodou por  1  timeslice, faltam  94
C  rodou por  1  timeslice, faltam  85
C  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  98
C  rodou por  1  timeslice, faltam  83
C  rodou por  1  timeslic

### Sobre STRIDE



1 - Repita o cenário da 2 de loteria e faça um novo gráfico.


In [1036]:
# STRIDE Questão 1

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 1] #tickets
escalonador_escolhido = "escalonador_STRIDE"
quantum = 1
seed = 0
simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslice, faltam  76
A  rodou por  1  timeslice, faltam  75
A  rodou por  1  timeslic

calcular unfairness e gerar o gráfico


In [1037]:
# stride Questão 1: Outras sementes

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 1] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 1
seed = 150

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslice, faltam  76
A  rodou por  1  timeslice, faltam  75
A  rodou por  1  timeslic

2 - No simulador maior, qual o efeito que ter muitos processos fazendo E/S tem no algoritmo? O que pode ser feito com a posição do processo quando ele volta de E/S?



In [1039]:
# stride Questão 2

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 1] #tickets
escalonador_escolhido = "escalonador_STRIDE"
quantum = 1
seed = 0

simulador_maior_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

Processo P1  chegou no tempo 0
A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
Processo P2  chegou no tempo 2
A  rodou por  1  timeslice, faltam  97
Processo P3  chegou no tempo 3
A  rodou por  1  timeslice, faltam  96
Processo P4  chegou no tempo 4
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
Processo P5  chegou no tempo 6
A  rodou por  1  timeslice, faltam  93
Processo P6  chegou no tempo 7
A  rodou por  1  timeslice, faltam  92
Processo P7  chegou no tempo 8
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
Processo P8  chegou no tempo 12
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
Processo P9  chegou no tempo 14
A  rodou por  1  timeslice, faltam  85
Processo P10  chegou no tempo 15
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timesl

3 - No simulador maior, rode um cenário igual para stride e loteria (com a mesma semente aleatória) e calcule tempo de resposta, de execução e __tempo de espera__ (tempo que o processo passou pronto). O que mudou entre os dois?

In [1040]:
# stride Questão 3: Outras sementes

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100] #tickets
escalonador_escolhido = "escalonador_LOTERIA"
quantum = 1
seed = 170

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
B  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
B  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslic

In [1041]:
# stride Questão 3: Outras sementes

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([100,100]) # Timeslices
billhetes = [100, 100] #tickets
escalonador_escolhido = "escalonador_STRIDE"
quantum = 1
seed = 170

simulador_simples_LOTERIA_STRIDE(escalonador_escolhido,nprocs, nomes, chanceio, tamanho, billhetes, quantum, seed)

A  rodou por  1  timeslice, faltam  99
A  rodou por  1  timeslice, faltam  98
A  rodou por  1  timeslice, faltam  97
A  rodou por  1  timeslice, faltam  96
A  rodou por  1  timeslice, faltam  95
A  rodou por  1  timeslice, faltam  94
A  rodou por  1  timeslice, faltam  93
A  rodou por  1  timeslice, faltam  92
A  rodou por  1  timeslice, faltam  91
A  rodou por  1  timeslice, faltam  90
A  rodou por  1  timeslice, faltam  89
A  rodou por  1  timeslice, faltam  88
A  rodou por  1  timeslice, faltam  87
A  rodou por  1  timeslice, faltam  86
A  rodou por  1  timeslice, faltam  85
A  rodou por  1  timeslice, faltam  84
A  rodou por  1  timeslice, faltam  83
A  rodou por  1  timeslice, faltam  82
A  rodou por  1  timeslice, faltam  81
A  rodou por  1  timeslice, faltam  80
A  rodou por  1  timeslice, faltam  79
A  rodou por  1  timeslice, faltam  78
A  rodou por  1  timeslice, faltam  77
A  rodou por  1  timeslice, faltam  76
A  rodou por  1  timeslice, faltam  75
A  rodou por  1  timeslic

### Sobre o MLFQ

1 - Rode o MLFQ no segundo simulador com duas filas e sem boost. O que aconteceu? 

2 - Quais configurações de parâmetros transformam o MLFQ no Round Robin?

3 - Rodando no primeiro simulador, faça 1 dos quatro processos ser 100\% CPU-Bound e os outros três 60\% I/O Bound. Use a configuração com 3 filas, com 5, 3 e 1 timeslices para as fila em ordem de priorida. De quanto tempo deve ser o Boost para que o processo CPU-Bound não sofra starvation e tenha CPU 1 vez a cada 20 timeslices?

4 - Teste diversas configurações de número de filas, quantum das filas e boost no simulador maior e ache a que leva ao melhor tempo de execução.


In [1038]:
# Criando processos para o MLFQ

nprocs = 2
nomes = ['A', 'B']
chanceio = [0, 0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([5,3]) # Timeslices
total = tamanho.sum()


procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,0,0)) #cria uma lista procs de Processos


# MLFQ
quantum = 1
tempoBloq = 1

escalonador = escalonador_MLFQ(procs) #troque escalonador pelo seu escalonador
bloqueados = []

tempo = 0
tempoE = []
tempoR = []
indice = 0
Iter = 0
teste = Processo('X',0,1,0,2)

random.seed(0)

print("Escalonador MLFQ")

while total>0:
    p = escalonador.proximo()
    if(p is not None):
        rodou, _ = p.roda(quantum) #adicione quantum como parâmetro, por enquanto nao temos E/S
        
        if len(tempoE) == 0:
            tempoE.append(rodou)
            tempoR.append(indice)
        else:
            tempoE.append(rodou+tempoE[indice])
            tempoR.append(tempoE[indice])
            indice += 1
            
        if(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        tempo+=rodou
        
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
    


print("Tempo de execução:", sum(tempoE) / nprocs)
print("Tempo de resposta:", sum(tempoR) / nprocs)
print("Tempo médio de execução:", sum(tempoE) / len(tempoE))
print("Tempo médio de resposta:", sum(tempoR) / len(tempoR))

Escalonador MLFQ
A  rodou por  1  timeslice, faltam  4
B  rodou por  1  timeslice, faltam  2
A  rodou por  1  timeslice, faltam  3
B  rodou por  1  timeslice, faltam  1
A  rodou por  1  timeslice, faltam  2
B  rodou por  1  timeslice, faltam  0
A  rodou por  1  timeslice, faltam  1
A  rodou por  1  timeslice, faltam  0
Tempo de execução: 18.0
Tempo de resposta: 14.0
Tempo médio de execução: 4.5
Tempo médio de resposta: 3.5
